In [40]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 필터 커널 세팅
kernel = np.ones((1, 1), np.uint8)
kernel2 = np.ones((3, 3), np.uint8)

############################################################################################################
    # 마지막 코드 #
############################################################################################################

# 영상 경로 설정
video_path= './test_data/video10.mp4'
cap = cv2.VideoCapture(video_path)

# 영상 이름
video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

# 필터 
red1 = np.array([0, 30, 10]) # 밝은 빨강(주황계열)
red2 = np.array([7, 250, 230])

red3 = np.array([155, 30, 10]) # 어두운 빨강(갈색계열)
red4 = np.array([180, 250, 230])

# 필터 커널 세팅
kernel = np.ones((2, 2), np.uint8)
kernel2 = np.ones((4, 4), np.uint8)

#frame_move = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)   )
#cap.set(cv2.CAP_PROP_POS_FRAMES, frame_move)
###################################################################################################################
while cap.isOpened() :
    #######################################################
    # 영상 읽기
    ret, img = cap.read()
    
    # 마지막 프레임 확인
    if ret is None:
        break
    img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2),  interpolation=cv2.INTER_AREA)
    
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    #yuv[yuv[:, :, 2]<170] = 0
    yuv[yuv[:, :, 2]<140] = 0
    yuv[yuv[:, :, 1]<110] = 0
    yuv[yuv[:, :, 0]>200] = 0
    img2 = yuv[ : , : , 2]
    cv2.imshow('img2', img2)
    # print(yuv[:, :, 2])
    

    thr, yuv_red = cv2.threshold(img2, 140, 255, cv2.THRESH_BINARY)
    
    cv2.imshow('yuv_red', yuv_red)
    erosion_mask = cv2.erode(yuv_red, kernel, iterations=1)
    dilation_mask = cv2.dilate(erosion_mask, kernel2, iterations=1) 


    edges = cv2.Canny(img, 100,200)
    
    dilation_mask2 = cv2.dilate(edges, kernel2, iterations=1) 
    erosion_mask2 = cv2.erode(dilation_mask2, kernel, iterations=1)

    res_red2 = cv2.bitwise_and(dilation_mask, dilation_mask, mask =dilation_mask2)
    
    cv2.imshow('res_red2', res_red2)
    
    _, contours, hierarchy = cv2.findContours(res_red2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    mask_external = np.zeros(res_red2.shape, res_red2.dtype)
    
    for i in range(len(contours)) :
        if hierarchy[0][i][3] == -1 :
            cv2.drawContours(mask_external, contours, i, 255, -1)

    mask_internal = np.zeros(res_red2.shape, res_red2.dtype)
    
    for i in range(len(contours)) :
        if hierarchy[0][i][3] != -1 :
            cv2.drawContours(mask_internal, contours, i, 255, -1)
                    
    _, _, stats_i, centroids_i = cv2.connectedComponentsWithStats(mask_internal)
    _, _, stats_e, centroids_e = cv2.connectedComponentsWithStats(mask_external)
    
    for idx, centroid_i in enumerate(centroids_i) :
        
        if stats_i[idx][0] == 0 and stats_i[idx][1] == 0 :
            continue
        if np.any(np.isnan(centroid_i)) :
            continue
            
        for centroid_e in centroids_e :
            if abs(centroid_i[0] - centroid_e[0]) < 5 and abs(centroid_i[1] - centroid_e[1]) < 5 :
                x, y, w, h, area = stats_i[idx]
                centerX, centerY = int(centroid_i[0]), int(centroid_i[1])

                if area > 120 and abs(w-h) < 5 :
                    try : 
                        detected_img = img[y-10 : y+h+10,  x-10 : x+w+10]
                        gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
                        circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=45,minRadius=7, maxRadius=40)
                        if circles is not None :
                            circles = np.uint16(np.around(circles))

                            for i in circles[0, :] :
                                cv2.circle(img, ( centerX, centerY ), 2, (0, 255, 0), 2)
                                cv2.rectangle(img, (x-10, y-10), (x+w+10, y+h+10), (0, 0, 255), 2)
                    except : 
                        continue


    cv2.imshow(video_file, img)

    
    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()

cap.release()
cv2.destroyAllWindows()

In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 필터 커널 세팅
kernel = np.ones((1, 1), np.uint8)
kernel2 = np.ones((3, 3), np.uint8)

############################################################################################################
    # 마지막 코드 #
############################################################################################################

# 영상 경로 설정
video_path= './test_data/new.mp4'
cap = cv2.VideoCapture(video_path)

# 영상 이름
video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

# 필터 
red1 = np.array([0, 30, 10]) # 밝은 빨강(주황계열)
red2 = np.array([7, 250, 230])

red3 = np.array([155, 30, 10]) # 어두운 빨강(갈색계열)
red4 = np.array([180, 250, 230])

# 필터 커널 세팅
kernel = np.ones((2, 2), np.uint8)
kernel2 = np.ones((4, 4), np.uint8)

#frame_move = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)   )
#cap.set(cv2.CAP_PROP_POS_FRAMES, frame_move)
###################################################################################################################
while cap.isOpened() :
    #######################################################
    # 영상 읽기
    ret, img = cap.read()
    
    # 마지막 프레임 확인
    if ret is None:
        break
        
    # 이미지 크기 변경
    img = cv2.resize(img, (img.shape[1] //2, img.shape[0]//2),  interpolation=cv2.INTER_AREA)
    
    # ROI 영역 추출
    new_img = np.zeros(img.shape, img.dtype)
    new_img[0 : img.shape[0]*2//3, : ] = img[0 : img.shape[0]*2//3 ,  : ]
    #cv2.imshow('new_img', new_img)
    
    # 1. 색 공간 변환 ( BGR -> YUV)
    yuv = cv2.cvtColor(new_img, cv2.COLOR_BGR2YUV)
    #yuv[yuv[:, :, 2]<170] = 0
    yuv[yuv[:, :, 2]<135] = 0
    yuv[yuv[:, :, 1]<110] = 0
    yuv[yuv[:, :, 0]>230] = 0
    img2 = yuv[ : , : , 2]
    
    # 스레드숄드 / 침식 / 팽창 적용 
    thr, yuv_red = cv2.threshold(img2, 140, 255, cv2.THRESH_BINARY)
    erosion_mask = cv2.erode(yuv_red, kernel, iterations=1)
    dilation_mask = cv2.dilate(erosion_mask, kernel2, iterations=1) 
    #cv2.imshow('dilation_mask', dilation_mask)
###########################################################################
    # 2. 캐니 엣지 
    edges = cv2.Canny(new_img, 100,200)
    
    # 침식 /팽창 적용
    dilation_mask2 = cv2.dilate(edges, kernel, iterations=1) 
    erosion_mask2 = cv2.erode(dilation_mask2, kernel, iterations=1)
    #cv2.imshow('erosion_mask2', erosion_mask2)
    
    #이미지 bitwise_and 연산
    res_red2 = cv2.bitwise_and(dilation_mask, dilation_mask, mask =erosion_mask2)
    #cv2.imshow('res_red2', res_red2)
    
###########################################################################
    ## 3. 컨투어 / 허프 원 추출 ##
    _, contours, hierarchy = cv2.findContours(res_red2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    mask_internal = np.zeros(res_red2.shape, res_red2.dtype)
    
    cv2.drawContours(mask_internal, contours, -1, 255, 2)

    cv2.imshow('mask_internal', mask_internal)
    
    _, _, stats_i, centroids_i = cv2.connectedComponentsWithStats(mask_internal)
    
    
    for idx, centroid_i in enumerate(centroids_i) :
        
        if stats_i[idx][0] == 0 and stats_i[idx][1] == 0 :
            continue
        if np.any(np.isnan(centroid_i)) :
            continue       
    
        x, y, w, h, area = stats_i[idx]
        centerX, centerY = int(centroid_i[0]), int(centroid_i[1])
        
        if area < 1700 and area > 200 and abs(w-h) < 5 :
            try : 
                detected_img = img[y+7 : y+h+7,  x+7 : x+w+7]
                gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
                circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=45,minRadius=5, maxRadius=38)
                if circles is not None :
                    circles = np.uint16(np.around(circles))
                    for i in circles[0, :] :
                        print(area)
                        cv2.circle(img, ( centerX, centerY ), 2, (0, 255, 0), 2)
                        cv2.rectangle(img, (x+7, y+7), (x+w+7, y+h+7), (0, 0, 255), 2)
            except : 
                continue

    
    cv2.imshow(video_file, img)

    
    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()

cap.release()
cv2.destroyAllWindows()